Age brackets for segmenting customers in the credit card or banking sector:
- 18-24: Young adults, often starting their financial journey.
- 25-34: Early career professionals, potentially with more significant financial responsibilities.
- 35-44: Mid-career individuals, often with established careers and possibly higher income.
- 45-54: Pre-retirement age, may have accumulated more wealth or financial assets.
- 55-64: Nearing retirement, often focusing on saving and investment.
- 65 and above: Retired or nearing retirement, may have different financial needs and behaviors.

In [86]:
import pandas as pd
import shutil


In [87]:
# Load CSV data
csv_file_path = 'UCI_Credit_Card.csv'
df = pd.read_csv(csv_file_path)

In [88]:
# Show summary of the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

In [89]:
# Investigate null values in each column
df.isnull().sum()

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default.payment.next.month    0
dtype: int64

In [90]:
# Define age bins and labels
bins = [18,25,35,45,55,65, float('inf')]
labels = ['18-24','25-34','35-44','45-54','55-64','65 and above']

# Original payment delay months
payment_delay_months = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']

# Corresponding month names
month_names = ['September', 'August', 'July', 'June', 'May', 'April']


# Create a dictionary for renaming the payment columns
rename_dict = dict(zip(payment_delay_months, month_names))

# Rename the columns in the DataFrame
df_renamed = df.rename(columns=rename_dict)

# Payment categories mapping
payment_categories = {
    -2: 'Paid Early',
    -1: 'Paid Duly',
    0: 'No Required Payment',
    1: '1 month',
    2: '2 months',
    3: '3 months',
    4: '4 months',
    5: '5 months',
    6: '6 months',
    7: '7 months',
    8: '8 months',
    9: '9 months or more'
}



In [91]:
# Add a new column 'Age_Bin' to the DataFrame
df_renamed['Age_Bin'] = pd.cut(df_renamed['AGE'], bins=bins, labels=labels, right=True)

In [92]:
# Initialize an empty DataFrame to hold the final data
final_df = pd.DataFrame()

Analyzing month and payment category based on Age Group

In [93]:
# Iterate over each month and payment category, count occurrences and add to the final DataFrame
for month in month_names:
    for value, category in payment_categories.items():
        # Count occurrences for each category in the current month
        count_df = df_renamed[df_renamed[month] == value].groupby('Age_Bin').size().reset_index(name=f'{month}_{category}')
        
        # Merge with the final DataFrame
        if final_df.empty:
            final_df = count_df
        else:
            final_df = final_df.merge(count_df, on='Age_Bin', how='outer')

final_df.head()

C:\Users\Ernie\AppData\Local\Temp\ipykernel_25472\784771552.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count_df = df_renamed[df_renamed[month] == value].groupby('Age_Bin').size().reset_index(name=f'{month}_{category}')
C:\Users\Ernie\AppData\Local\Temp\ipykernel_25472\784771552.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count_df = df_renamed[df_renamed[month] == value].groupby('Age_Bin').size().reset_index(name=f'{month}_{category}')
C:\Users\Ernie\AppData\Local\Temp\ipykernel_25472\784771552.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a fu

,Age_Bin,September_Paid Early,September_Paid Duly,September_No Required Payment,September_1 month,September_2 months,September_3 months,September_4 months,September_5 months,September_6 months,...,April_No Required Payment,April_1 month,April_2 months,April_3 months,April_4 months,April_5 months,April_6 months,April_7 months,April_8 months,April_9 months or more
0,18-24,147,510,2186,475,466,59,21,3,1,...,2415,0,459,45,14,3,5,7,0,0
1,25-34,1209,2449,6483,1616,1013,125,21,9,6,...,7051,0,1168,59,19,5,8,14,1,0
2,35-44,934,1853,3864,1020,726,77,22,11,0,...,4358,0,706,55,4,1,4,13,0,0
3,45-54,386,728,1822,460,369,45,8,2,3,...,2015,0,353,16,7,3,1,11,1,0
4,55-64,74,133,340,108,80,16,4,1,1,...,399,0,71,9,4,1,0,1,0,0


In [94]:
# Define the path to save the CSV file
csv_file_path = 'final_df.csv'

# Save the DataFrame to CSV
final_df.to_csv(csv_file_path, index=True)

In [95]:
# Define the path to save the JSON file
json_file_path = 'final_df.json'

# Convert the DataFrame to JSON
final_df.to_json(json_file_path, orient='records')

# Load and check the JSON file
with open(json_file_path, 'r') as f:
    json_data = f.read()

print(json_data[:500])

[{"Age_Bin":"18-24","September_Paid Early":147,"September_Paid Duly":510,"September_No Required Payment":2186,"September_1 month":475,"September_2 months":466,"September_3 months":59,"September_4 months":21,"September_5 months":3,"September_6 months":1,"September_7 months":0,"September_8 months":3,"September_9 months or more":0,"August_Paid Early":225,"August_Paid Duly":524,"August_No Required Payment":2382,"August_1 month":3,"August_2 months":636,"August_3 months":73,"August_4 months":19,"Augus


Creating a multi-index DataFrame

In [96]:
# Convert 'Age_Bin' to a string type if it is categorical
final_df['Age_Bin'] = final_df['Age_Bin'].astype(str)

In [97]:
# Fill NaN values with 0 (since we're counting occurrences)
final_df = final_df.fillna(0)

In [98]:
# Set 'Age_Bin' as the index
final_df = final_df.set_index('Age_Bin')

In [99]:
# Create MultiIndex for the columns
multi_index_columns = pd.MultiIndex.from_product([month_names, list(payment_categories.values())], names=['Month', 'Category'])


In [100]:
# Rename columns with MultiIndex
final_df.columns = multi_index_columns
final_df

Month         September                                                 \
Category     Paid Early Paid Duly No Required Payment 1 month 2 months   
Age_Bin                                                                  
18-24               147       510                2186     475      466   
25-34              1209      2449                6483    1616     1013   
35-44               934      1853                3864    1020      726   
45-54               386       728                1822     460      369   
55-64                74       133                 340     108       80   
65 and above          9        13                  42       9       13   

Month                                                      ...  \
Category     3 months 4 months 5 months 6 months 7 months  ...   
Age_Bin                                                    ...   
18-24              59       21        3        1        0  ...   
25-34             125       21        9        6        2  ...   
35-44              77       22       11        0        6  ...   
45-54              45        8        2        3        1  ...   
55-64              16        4        1        1        0  ...   
65 and above        0        0        0        0        0  ...   

Month                      April                                              \
Category     No Required Payment 1 month 2 months 3 months 4 months 5 months   
Age_Bin                                                                        
18-24                       2415       0      459       45       14        3   
25-34                       7051       0     1168       59       19        5   
35-44                       4358       0      706       55        4        1   
45-54                       2015       0      353       16        7        3   
55-64                        399       0       71        9        4        1   
65 and above                  48       0        9        0        1        0   

Month                                                     
Category     6 months 7 months 8 months 9 months or more  
Age_Bin                                                   
18-24               5        7        0                0  
25-34               8       14        1                0  
35-44               4       13        0                0  
45-54               1       11        1                0  
55-64               0        1        0                0  
65 and above        1        0        0                0  

[6 rows x 72 columns]

Code verification

In [101]:
 
# Load CSV data
csv_file_path = 'UCI_Credit_Card.csv'
df = pd.read_csv(csv_file_path)

In [102]:
# Define age bins and labels
bins = [18,25,35,45,55,64, float('inf')]
labels = ['18-24','24-34','35-44','45-54','55-64','65 and above']
df['Age_Bin'] = pd.cut(df['AGE'], bins=bins, labels=labels, right=True)

# Filter for the specific Age_Bin '35-44'
age_bin_filtered_df = df[df['Age_Bin'] == '35-44']

# Count occurrences of the value 7 in 'PAY_0'
count_value_7 = (age_bin_filtered_df['PAY_0'] == -2).sum()

print("Count of value 7 in 'PAY_0' for 'Age_Bin' 35-44:", count_value_7)

Count of value 7 in 'PAY_0' for 'Age_Bin' 35-44: 990


Analyzing Education and Payment Group based on Age group

In [103]:
# Load CSV data
csv_file_path = 'UCI_Credit_Card.csv'
df = pd.read_csv(csv_file_path)

In [104]:
# Define the columns for billing amounts and payment amounts
bill_columns = ['BILL_AMT6', 'BILL_AMT5', 'BILL_AMT4', 'BILL_AMT3', 'BILL_AMT2','BILL_AMT1']
pay_columns = ['PAY_AMT6', 'PAY_AMT5', 'PAY_AMT4', 'PAY_AMT3', 'PAY_AMT2', 'PAY_AMT1']

In [105]:
# Calculate the outstanding amount paid for each month
for i in range(len(bill_columns)):
    df[f'Outstanding_Amt_Month_{6-i}'] = df[bill_columns[i]] - df[pay_columns[i]]

# Display the first few rows
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month,Outstanding_Amt_Month_6,Outstanding_Amt_Month_5,Outstanding_Amt_Month_4,Outstanding_Amt_Month_3,Outstanding_Amt_Month_2,Outstanding_Amt_Month_1
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,1,0.0,0.0,0.0,689.0,2413.0,3913.0
1,2,120000.0,2,2,2,26,-1,2,0,0,...,1000.0,0.0,2000.0,1,1261.0,3455.0,2272.0,1682.0,725.0,2682.0
2,3,90000.0,2,2,2,34,0,0,0,0,...,1000.0,1000.0,5000.0,0,10549.0,13948.0,13331.0,12559.0,12527.0,27721.0
3,4,50000.0,2,2,1,37,0,0,0,0,...,1100.0,1069.0,1000.0,0,28547.0,27890.0,27214.0,48091.0,46214.0,44990.0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,9000.0,689.0,679.0,0,18452.0,18457.0,11940.0,25835.0,-31011.0,6617.0


In [106]:
# Define age bins and labels
bins = [18,24,34,44,54,64, float('inf')]
labels = ['18-24','24-34','35-44','45-54','55-64','65 and above']
df['Age_Bin'] = pd.cut(df['AGE'], bins=bins, labels=labels, right=True)


In [107]:
# Calculate the mean of Outstanding amount for each Age Group
age_group_outstanding = df.groupby('Age_Bin')[[f'Outstanding_Amt_Month_{6-i}' for i in range(len(bill_columns))]].mean()

# Round the result to 2 decimal places
age_group_outstanding = age_group_outstanding.round(2)

age_group_outstanding.head()

C:\Users\Ernie\AppData\Local\Temp\ipykernel_25472\173873143.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_group_outstanding = df.groupby('Age_Bin')[[f'Outstanding_Amt_Month_{6-i}' for i in range(len(bill_columns))]].mean()


,Outstanding_Amt_Month_6,Outstanding_Amt_Month_5,Outstanding_Amt_Month_4,Outstanding_Amt_Month_3,Outstanding_Amt_Month_2,Outstanding_Amt_Month_1
Age_Bin,,,,,,
18-24,19200.18,20032.67,21969.32,24874.32,25876.92,26889.65
24-34,34577.70,36472.29,39375.53,42635.51,44077.58,46424.71
35-44,36186.99,38366.78,41421.42,44482.82,45915.36,48432.42
45-54,33927.38,35580.09,38576.63,43094.54,44779.38,47638.70
55-64,33274.71,36022.23,39831.66,43669.19,45682.02,46688.77


In [108]:
age_group_outstanding.rename(columns = {
    'Outstanding_Amt_Month_6' : 'April',
    'Outstanding_Amt_Month_5': 'May',
    'Outstanding_Amt_Month_4': 'June',
    'Outstanding_Amt_Month_3': 'July',
    'Outstanding_Amt_Month_2': 'August',
    'Outstanding_Amt_Month_1': 'September'
}, inplace=True)
age_group_outstanding.head()

,April,May,June,July,August,September
Age_Bin,,,,,,
18-24,19200.18,20032.67,21969.32,24874.32,25876.92,26889.65
24-34,34577.70,36472.29,39375.53,42635.51,44077.58,46424.71
35-44,36186.99,38366.78,41421.42,44482.82,45915.36,48432.42
45-54,33927.38,35580.09,38576.63,43094.54,44779.38,47638.70
55-64,33274.71,36022.23,39831.66,43669.19,45682.02,46688.77


In [109]:
# Convert the DataFrame to a JSON format
age_group_outstanding_json = age_group_outstanding.reset_index().to_json(orient='records')

# Save the JSON data to a file
with open('age_group_outstanding.json','w') as f:
    f.write(age_group_outstanding_json)

Credit Utilization Ratio (Outstanding Balance / Credit limit * 100%) 

In [110]:
# Load CSV data
csv_file_path = 'UCI_Credit_Card.csv'
df = pd.read_csv(csv_file_path)

In [111]:
# Define the columns for billing amounts and payment amounts
bill_columns = ['BILL_AMT6', 'BILL_AMT5', 'BILL_AMT4', 'BILL_AMT3', 'BILL_AMT2', 'BILL_AMT1']
pay_columns = ['PAY_AMT6', 'PAY_AMT5', 'PAY_AMT4', 'PAY_AMT3', 'PAY_AMT2', 'PAY_AMT1']

In [112]:
# Calculate the outstanding amount paid for each month
for i in range(len(bill_columns)):
    df[f'Outstanding_Amt_Month_{6-i}'] = df[bill_columns[i]] - df[pay_columns[i]]

In [113]:
# Define the columns that hold the outstanding amounts
outstanding_columns = [f'Outstanding_Amt_Month_{6-i}' for i in range(len(bill_columns))]

In [114]:
# Define the credit limit column
credit_limit_column = 'LIMIT_BAL'


In [115]:
# Calculate the credit utilization ratio for each month and round to 2 decimal places
for i, col in enumerate(outstanding_columns):
    df[f'CUR_Mth_{6-i}(%)'] = (df[col] / df[credit_limit_column] * 100).round(2)
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,Outstanding_Amt_Month_4,Outstanding_Amt_Month_3,Outstanding_Amt_Month_2,Outstanding_Amt_Month_1,CUR_Mth_6(%),CUR_Mth_5(%),CUR_Mth_4(%),CUR_Mth_3(%),CUR_Mth_2(%),CUR_Mth_1(%)
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,689.0,2413.0,3913.0,0.00,0.00,0.00,3.44,12.06,19.56
1,2,120000.0,2,2,2,26,-1,2,0,0,...,2272.0,1682.0,725.0,2682.0,1.05,2.88,1.89,1.40,0.60,2.24
2,3,90000.0,2,2,2,34,0,0,0,0,...,13331.0,12559.0,12527.0,27721.0,11.72,15.50,14.81,13.95,13.92,30.80
3,4,50000.0,2,2,1,37,0,0,0,0,...,27214.0,48091.0,46214.0,44990.0,57.09,55.78,54.43,96.18,92.43,89.98
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,11940.0,25835.0,-31011.0,6617.0,36.90,36.91,23.88,51.67,-62.02,13.23


In [116]:
# Define age bins and labels
bins = [18,24,34,44,54,64, float('inf')]
labels = ['18-24','25-34','35-44','45-54','55-64','65 and above']
df['Age_Bin'] = pd.cut(df['AGE'], bins=bins, labels=labels, right=True)

In [117]:
# Calculate the mean of CUR for each age group
# Select all CUR columns
mean_CUR_columns = [f'CUR_Mth_{6-i}(%)' for i in range(len(bill_columns))]
age_group_CUR = df.groupby('Age_Bin')[mean_CUR_columns].mean().round(2)

# Display the results
print(age_group_CUR)


              CUR_Mth_6(%)  CUR_Mth_5(%)  CUR_Mth_4(%)  CUR_Mth_3(%)  \
Age_Bin                                                                
18-24                39.21         41.44         44.68         48.68   
25-34                27.65         29.08         31.45         33.87   
35-44                25.80         27.29         29.65         32.34   
45-54                28.09         29.71         32.46         37.01   
55-64                30.54         32.14         35.28         39.75   
65 and above         36.95         36.69         40.46         41.53   

              CUR_Mth_2(%)  CUR_Mth_1(%)  
Age_Bin                                   
18-24                49.66         50.56  
25-34                35.09         36.21  
35-44                33.63         34.83  
45-54                38.47         40.18  
55-64                41.24         42.48  
65 and above         43.83         44.41  


C:\Users\Ernie\AppData\Local\Temp\ipykernel_25472\1434849011.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_group_CUR = df.groupby('Age_Bin')[mean_CUR_columns].mean().round(2)
